## Ingesting PDF

In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [22]:
import nltk
nltk.download('punkt_tab') # for tokenization
nltk.download('averaged_perceptron_tagger_eng') # for POS tagging

[nltk_data] Downloading package punkt_tab to /home/supun/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/supun/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [4]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader


from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

In [5]:
%pwd

'/media/supun/Learning Hub/AI & Machine Learning/WORKING..../Mini project 04-chatbot/ollama_pdf_rag'

In [6]:
%pip install pytesseract # OCR, Optical Character Recognition
%pip install tesseract-ocr

Note: you may need to restart the kernel to use updated packages.
  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Preparing metadata (setup.py) ... done
  Using cached Cython-3.0.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
Using cached Cython-3.0.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [32 lines of output]
      /home/supun/anaconda3/envs/ollamachatbot/lib/python3.10/site-packages/setuptools/dist.py:447: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              By 2024-Sep-26, you need to update

In [7]:
# local_path = "data/WEF_The_Global_Cooperation_Barometer_2024.pdf"

# # Local PDF file uploads
# if local_path:
# #   loader = UnstructuredPDFLoader(file_path=local_path)
#     loader=DirectoryLoader(file_path=local_path,
#                     glob="*.pdf",
#                     loader_cls=PyPDFLoader)

#     data=loader.load()
# else:
#   print("Upload a PDF file")


local_path = "data/WEF_The_Global_Cooperation_Barometer_2024.pdf"

# Local PDF file upload
if local_path:
    # Use PyPDFLoader to load the specific PDF file
    loader = PyPDFLoader(local_path)
    data = loader.load()
else:
    print("Upload a PDF file")

In [8]:
# Preview first page
data[10].page_content

'The innovation and technology pillar examines \nhow global cooperation, through exchanges of \nknowledge and people, accelerates innovation and \ncreates beneficial technological progress. \nFrom 2012-2020, innovation and technology \ncooperation maintained strong and significant \ngrowth across most barometer metrics (particularly in cross-border data flows and IT services trade). \nSince 2020, cooperation has leveled off, with \ncritical aspects starting to decline (e.g. cross-\nborder patent applications and international student \nflows) (Figure 4). Innovation and technology\nPillar 2: Innovation and technology FIGURE 4\n2012-2022: Significant, sustained \nincrease arrested by the pandemic \nGlobal cooperation in innovation and technology \nadvanced significantly from 2012-2020, likely \nbecause these years saw a significant shift toward \ndigital services, including cloud computing and \napplied AI. Rapid growth in technological streams \nsuch as intellectual property and data fa

## Vector Embeddings

In [14]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [17]:
!ollama pull nomic-embed-text # pull the nomic-embed-text model

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90... 

In [1]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED           
nomic-embed-text:latest	0a109f422b47	274 MB	About a minute ago	


In [8]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [10]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [15]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 26/26 [02:40<00:00,  6.17s/it]


## Retrieval

In [16]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [17]:
# LLM from Ollama
# local_model = "mistral"
local_model = "moondream"
llm = ChatOllama(model=local_model)

#### ollama pull mistral <<-- run this command to pull the model in the terminal

In [18]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [19]:
# MultiQueryRetriever, combining vector database and LLM
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt,
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [20]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
chain.invoke(input("")) # Ask a question,

In [23]:
chain.invoke("What are the 5 pillars of global cooperation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.47it/s]


'\nThe five pillars of global cooperation are trade and capital, innovation and technology, climate and natural capital, health and wellness, and peace and security. These pillars represent areas where countries can work together to achieve mutual benefits and address common challenges. The Global Cooperation Barometer measures the level of cooperation in these areas over time, providing insights into how well nations collaborate on various issues.'

In [ ]:
# # Delete all collections in the db
# vector_db.delete_collection()